In [28]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

In [3]:
ucl = pd.read_excel('UCL_playerstats_orig.xlsx')

In [5]:
ucl.head()

,PlayerID,Player,Position,Age,Nationality,Team,Competitions,Phase,Phase_Tier,Value10^6,...,Balls_recovered,Tackles_Won,Tackles_Lost,Saves,Goals_Conceded,Clean_Sheets,MOTM_Awards,Minutes_played,Match_played,DNP
0,1,Theo Hernandez,Defender,27,France,Milan,Serie A,Knockout_phase,D,40.0,...,53,2,5,0,0,0,0,861,10,0
1,2,Nicolo Barella,Midfielder,28,Italy,Inter,Serie A,Final,S,80.0,...,32,8,15,0,0,0,2,994,13,2
2,3,Lewandowski,Forward,36,Poland,Barcelona,La Liga,Semi_finals,A,15.0,...,11,4,1,0,0,0,2,985,13,1
3,4,Raphinha,Forward,28,Brazil,Barcelona,La Liga,Semi_finals,A,80.0,...,18,2,7,0,0,0,3,1225,14,0
4,5,Van Dijk,Defender,33,Netherlands,Liverpool FC,Premier League,Round16,C,50.0,...,51,3,3,0,0,0,0,840,9,1


#### HAPI 1: Krijimi i target-it “elite”
##### Do të klasifikojmë lojtarët në dy grupe: Elite Player (1)  Jo-Elite Player (0) 

In [12]:
ucl['Elite'] = ucl['Rating'].apply(lambda x: 1   if x > 7.3 else 0)

# Kontrollo sa lojtarë janë elite dhe sa jo
print(ucl['Elite'].value_counts())

Elite
0    146
1     64
Name: count, dtype: int64


In [13]:
ucl[['Player', 'Rating', 'Elite']].head(10)

,Player,Rating,Elite
0,Theo Hernandez,7.02,0
1,Nicolo Barella,7.11,0
2,Lewandowski,7.48,1
3,Raphinha,8.31,1
4,Van Dijk,7.31,1
5,Jude Bellingham,7.60,1
6,Odegaard,7.31,1
7,Brahim Diaz,6.93,0
8,Hans Vanaken,7.18,0
9,Otamendi,7.17,0


#### HAPI 2: Përgatitja e të dhënave për klasifikim

In [14]:
target = 'Elite'

features = [
    'Age',
    'Position',
    'Team',
    'Goals',
    'Assists',
    'Total_attempts',
    'Minutes_played',
    'MOTM_Awards'
]

In [16]:
features

['Age',
 'Position',
 'Team',
 'Goals',
 'Assists',
 'Total_attempts',
 'Minutes_played',
 'MOTM_Awards']

#### HAPI 3: Ndarja e input (X) dhe target (y)

In [17]:
X = ucl[features]
y = ucl[target]

In [19]:
X

,Age,Position,Team,Goals,Assists,Total_attempts,Minutes_played,MOTM_Awards
0,27,Defender,Milan,0,0,17,861,0
1,28,Midfielder,Inter,0,2,12,994,2
2,36,Forward,Barcelona,11,0,34,985,2
3,28,Forward,Barcelona,13,9,52,1225,3
4,33,Defender,Liverpool FC,1,0,4,840,0
...,...,...,...,...,...,...,...,...
205,22,Forward,Shakhtar Donetsk,2,0,10,460,0
206,35,Midfielder,Lille,1,1,9,388,0
207,27,Midfielder,Juventus,0,0,2,673,0
208,26,Midfielder,Bayer Leverkusen,0,0,10,666,0


#### 3️⃣ One-Hot Encoding për kolonat kategorike

In [21]:
X = pd.get_dummies(X, columns=['Position', 'Team'], drop_first=True)

#### 4️⃣ Ndarja train/test

In [22]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

#### 5️⃣ Trajnimi i Decision Tree Classifier

In [23]:
from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)

,criterion,'gini'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,42
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


#### 📊6️⃣: Vlerësimi i modelit

In [24]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Predikimi
dt_preds = dt_model.predict(X_test)

In [25]:
# Metrikat
print("Accuracy :", round(accuracy_score(y_test, dt_preds), 2))
print("Precision:", round(precision_score(y_test, dt_preds), 2))
print("Recall   :", round(recall_score(y_test, dt_preds), 2))
print("F1-Score :", round(f1_score(y_test, dt_preds), 2))

Accuracy : 0.7
Precision: 0.48
Recall   : 0.67
F1-Score : 0.56


In [27]:
# Confusion Matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, dt_preds))


Confusion Matrix:
[[43 17]
 [ 8 16]]
